<a href="https://colab.research.google.com/github/liuy01510/portfolio/blob/master/Optimization_I_Sudoku_Solver_PF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction & Objectives
- Objective: Creating a Sudoku solver that solves the puzzle using LP tools.
- Requirements: 
    - Determining the type of solver to be used.
    - Modelling the constraints in terms of the LP software.


# Importing the required modules.
- In this script, the Google's ortools modules will be the LP solver used.

In [0]:
# Checking if the ortools module has been installed, else install it.
try:
    import ortools
except:
    !pip install ortools
    import ortools

# Importing the remaining functions
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from collections import OrderedDict
import time

# Sample sudoku problem
- The sudoku puzzle will be pulled using an API from a random sudoku puzzle generator.

In [20]:
url=f'https://sugoku.herokuapp.com/board?difficulty=hard'
responseObj=requests.get(url) # getting the puzzle from the server.

# Opening the response object and creating the puzzle
responseObj=responseObj.content # returns a json puzzle 
responseObj=json.loads(responseObj) #loading the byte json file into a python dict object.
v=OrderedDict() # create a new dictionary to handle all the variables presents within this script.
v['given board']=np.array(responseObj['board']) # casting the board to a numpy array object.
v['board constraints']=[(x,y,v['given board'][x][y]) for x in range(9) for y in range(9)\
                        if v['given board'][x][y]!=0] # given board constraints.
print("The given board is shown as follows:")
print(v['given board'])

The given board is shown as follows:
[[0 0 0 9 0 0 0 0 0]
 [0 0 6 0 2 7 0 0 9]
 [7 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 6]
 [0 4 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 2 0 0]
 [0 3 5 0 9 4 8 0 1]
 [0 6 0 5 0 2 0 4 3]
 [9 0 0 0 3 1 0 6 2]]


# Solving the problem
- The sudoku puzzle can be expressed in the form of a constraint programming problem.
- Therefore, the constraint programming solver (CP-SAT) will be used to solve this puzzle.

In [21]:
# Importing the solver
v['start time']=time.time()
from ortools.sat.python import cp_model
model=cp_model.CpModel()
solver=cp_model.CpSolver()

# Creating the variables present in this puzzle
for x in range(9):
    for y in range(9):
        v[f"x{x}y{y}"]=model.NewIntVar(1,9,f'x{x}y{y}')

# Setting the constraints present in this problem

## all different numbers for each row
for x in range(9):
    model.AddAllDifferent([v[f"x{x}y{y}"] for y in range(9)])

## all different numbers for each column
for y in range(9):
    model.AddAllDifferent([v[f"x{x}y{y}"] for x in range(9)])

## all different numbers for each box
for x in range(0,9,3):
    for y in range(0,9,3):
        model.AddAllDifferent([v[f"x{i}y{j}"] for i in range(x,x+3) for j in range(y,y+3)])

## constraints provided by the puzzle
for x,y,val in v['board constraints']:
    model.Add(v[f"x{x}y{y}"]==val)

# Solving the problem
solution=solver.Solve(model)
v['end time']=time.time()
if solution==cp_model.FEASIBLE:
    print('A feasible solution has been found!')
    v['result board']=np.zeros((9,9),dtype=np.int8)
    for x in range(9):
        for y in range(9):
            v['result board'][x][y]=solver.Value(v[f"x{x}y{y}"])
    print('The solved sudoku puzzle is seen below:')
    print(v['result board'])
    print(f"The sudoku puzzle was solved in {v['end time']-v['start time']} seconds.")
else:
    print('The solver failed to solve the Sudoku problem.')



A feasible solution has been found!
The solved sudoku puzzle is seen below:
[[4 1 3 9 5 8 6 2 7]
 [8 5 6 1 2 7 4 3 9]
 [7 2 9 3 4 6 1 8 5]
 [3 8 2 4 1 5 7 9 6]
 [5 4 7 2 6 9 3 1 8]
 [6 9 1 7 8 3 2 5 4]
 [2 3 5 6 9 4 8 7 1]
 [1 6 8 5 7 2 9 4 3]
 [9 7 4 8 3 1 5 6 2]]
The sudoku puzzle was solved in 0.01010751724243164 seconds.
